- Model Selection

In [1]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('Humor_var.db')
cur = conn.cursor()
df = pd.read_sql('SELECT * FROM var', conn)
#df['new'] = np.zeros_like(range(len(df)))

In [55]:
pipe = Pipeline([('pre',StandardScaler()), ('clf',KNeighborsClassifier())])

In [56]:
grid = GridSearchCV(pipe, [{'pre':[StandardScaler(), MinMaxScaler()]},
                          {'clf':[KNeighborsClassifier()], 'clf__n_neighbors':range(2,8)},
                          {'clf':[LogisticRegression()], 'clf__penalty':['l1','l2'],'clf__C':[0.1,1,10,100,1000], 'clf__solver':['liblinear','sag','saga','lbfgs']},
                          {'clf':[SVC()], 'clf__C':[0.1,1,10,100,1000], 'clf__gamma':[0.1,1,10], 'clf__kernel':['rbf','linear','poly','sigmoid']},
                          {'clf':[DecisionTreeClassifier()], 'clf__criterion':['gini','entropy'], 'clf__max_depth':range(4,20), 'clf__min_samples_split':range(2,6), 'clf__min_samples_leaf':range(2,4), 'clf__random_state':[0,13]},
                          {'clf':[RandomForestClassifier()], 'clf__n_estimators':[10,100]}])

In [49]:
grid.fit(df.iloc[:,1:-1], df.iloc[:,-1])
pd.DataFrame(grid.cv_results_).T

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1356, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 926, in _fit_liblinear
    raise ValueError("This solver needs samples of at least 2 classes"
ValueError: This solver needs samples of

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1356, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 926, in _fit_liblinear
    raise ValueError("This solver needs samples of at least 2 classes"
ValueError: This solver needs samples of

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1372, in fit
    raise ValueError("This solver needs samples of at least 2 classes"
ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/gozj3319/anaconda3/lib

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver 

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver 

...

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 164, in fit
    y = self._validate_targets(y)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 549, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

  warnings.warn("Estimator fit failed. The score o

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 164, in fit
    y = self._validate_targets(y)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 549, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

  warnings.warn("Estimator fit failed. The score o

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 164, in fit
    y = self._validate_targets(y)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 549, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

  warnings.warn("Estimator fit failed. The score o

/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 164, in fit
    y = self._validate_targets(y)
  File "/home/gozj3319/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 549, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

  warnings.warn("Estimator fit failed. The score o

GridSearchCV(estimator=Pipeline(steps=[('pre', StandardScaler()),
                                       ('clf', KNeighborsClassifier())]),
             param_grid=[{'pre': [StandardScaler(), MinMaxScaler()]},
                         {'clf': [KNeighborsClassifier()],
                          'clf__n_neighbors': range(2, 8)},
                         {'clf': [LogisticRegression()],
                          'clf__C': [0.1, 1, 10, 100, 1000],
                          'clf__penalty': ['l1', 'l2'],
                          'clf__solver': ['liblinear', 'sag', 'saga', 'lbfgs']},
                         {'clf': [SVC()], 'clf__C': [0.1, 1, 10, 100, 1000],
                          'clf__gamma': [0.1, 1, 10],
                          'clf__kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
                         {'clf': [DecisionTreeClassifier()],
                          'clf__criterion': ['gini', 'entropy'],
                          'clf__max_depth': range(4, 20),
                     

In [ ]:
# VotingClassifier([('Model1', LogisticRegression()), ('Model2', 등등)])

In [ ]:
# XGBoosting

- Feature Selection

In [66]:
from sklearn.feature_selection import RFE

In [ ]:
# RFE()